In [1]:
import os
import math

import cv2
import numpy as np
import paddle

from paddleseg import utils
from paddleseg.core import infer
from paddleseg.utils import logger, progbar, visualize, get_image_list
from paddleseg.cvlibs import manager
from paddleseg.transforms import Compose

In [2]:
model = manager.MODELS.components_dict['SegFormer_B5'](num_classes=20)
model_path = './best_model/model.pdparams'

W0308 17:34:29.559701  8275 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 11.2
W0308 17:34:29.562285  8275 device_context.cc:465] device: 0, cuDNN Version: 7.6.
W0308 17:34:30.524646  8275 device_context.h:397] WARNING: device: 0. The installed Paddle is compiled with CUDNN 8.1, but CUDNN version in your machine is 7.6, which may cause serious incompatible bug. Please recompile or reinstall Paddle with compatible CUDNN version.


In [3]:
transforms = Compose([manager.TRANSFORMS['Normalize']()])
transforms

In [4]:
image_list, image_dir = get_image_list('../datasets/test/image/')

In [5]:
image_list, image_dir

(['../datasets/test/image/08909_00.jpg', '../datasets/test/image/m_00002.jpg'],
 '../datasets/test/image/')

In [6]:
import pickle
with open('../colormap.pkl', 'rb') as file:
    color_map = pickle.load(file)

In [7]:
image_list

['../datasets/test/image/08909_00.jpg', '../datasets/test/image/m_00002.jpg']

In [8]:
# 高层 Paddleseg API
from paddleseg.core import predict
predict(model, 
        model_path=model_path, 
        transforms=transforms, 
        image_list=image_list, 
        image_dir=image_dir, 
        save_dir='output',
        custom_color=color_map)

2022-03-08 17:34:32 [INFO]	Loading pretrained model from ./best_model/model.pdparams
2022-03-08 17:34:32 [INFO]	There are 1067/1067 variables loaded into SegFormer.
2022-03-08 17:34:32 [INFO]	Start to predict...


2/2 [==============================] - 0s 232ms/step


In [9]:
image_list

['../datasets/test/image/08909_00.jpg', '../datasets/test/image/m_00002.jpg']

In [22]:
from pathlib import Path
from PIL import Image

# 仅做 Inference
utils.utils.load_entire_model(model, model_path)
model.eval()

progbar_pred = progbar.Progbar(target=len(image_list), verbose=1)

out_img_dir = Path('./output/res')

logger.info("start to predict...")
with paddle.no_grad():
    for i, im_path in enumerate(image_list):
        print(i, im_path)
        im = cv2.imread(im_path)
        ori_shape = im.shape[:2]
        im, _ = transforms(im)
        im = im[np.newaxis, ...]
        im = paddle.to_tensor(im)
        
        pred, _ = infer.inference(model, 
                                  im, 
                                  ori_shape=ori_shape, 
                                  transforms=transforms.transforms, 
                                  is_slide=False, 
                                  stride=False, 
                                  crop_size=None)
        pred = paddle.squeeze(pred)
        pred = pred.numpy().astype('uint8')
        
        # 输出添加调色板信息的分割图
        img_name = Path(im_path).stem
        pred_mask = Image.fromarray(pred, mode='P')
        pred_mask.putpalette(color_map)
        out_img_dir.mkdir(parents=True, exist_ok=True)
        pred_mask.save(f'{out_img_dir.joinpath(img_name)}.png')

2022-03-08 17:48:25 [INFO]	Loading pretrained model from ./best_model/model.pdparams
2022-03-08 17:48:25 [INFO]	There are 1067/1067 variables loaded into SegFormer.
2022-03-08 17:48:25 [INFO]	start to predict...
0 ../datasets/test/image/08909_00.jpg
1 ../datasets/test/image/m_00002.jpg
